In [1]:
%load_ext autoreload
%autoreload 2

# Import Lib

In [2]:
from ezorm.variables import EzORM, Field, Optional, Type
from ezorm.crud import Create, Read, Update, Delete
from ezorm.engine import duck_engine
# from ezorm.configuration import settings
from ezorm.db_management import create_tables, delete_tables
from ezorm import ezo

In [3]:
ezo.configure(db=duck_engine)

# Create data model

In [4]:
class DPoPModel(EzORM):
    jti:str = Field(description="your unique jti", default=None)

class AccessTokenModel(DPoPModel):
    access_token:str = Field(description="access token generated by server", default=None)
    client_id:str = Field(description="client it retrieved from client", default=None)
    exp:int = Field(description="expiration on access token should be less than an hour", default=None)
    active:bool = Field(description="true if a record is still valid, otherwise false", default=None)
    remark:Optional[str] = Field(description="reason why each record changes active from true to false", default=None)

class RefreshTokenModel(AccessTokenModel):
    refresh_token:str = Field(description="expiration varied from token type", default=None)
    exp:int = Field(description="expiration on access token should be at least a day", default=None)

tables = [DPoPModel, AccessTokenModel, RefreshTokenModel]

# Create tables

In [5]:
create_tables(tables)

Model: dpopmodel created successfully
Model: accesstokenmodel created successfully
Model: refreshtokenmodel created successfully
All tables created successfully


In [6]:
query = """\
SELECT * 
FROM information_schema.tables 
WHERE table_schema = 'main';
""".strip()
duck_engine(query, [])

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,ezorm,main,accesstokenmodel,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,ezorm,main,dpopmodel,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,ezorm,main,refreshtokenmodel,BASE TABLE,None,None,None,None,None,YES,NO,None,None


# Create record

In [7]:
Create(DPoPModel(jti="testdpop1"))

,Count
0,1


In [8]:
Create(DPoPModel(jti="testdpop2"))

,Count
0,1


# Read record

In [9]:
Read(DPoPModel(jti="testdpop1"))

,jti
0,testdpop1


In [10]:
Read(DPoPModel(jti="testdpop2"))

,jti
0,testdpop2


# Update record

In [11]:
existing = DPoPModel(jti="testdpop2")
new = DPoPModel(jti="testdpop3")
Update(existing, new)

,Count
0,1


In [12]:
Read(existing)

,jti


In [13]:
Read(new)

,jti
0,testdpop3


# Delete record

In [14]:
Delete(DPoPModel(jti="testdpop3"))

,Count
0,1


In [15]:
Read(DPoPModel(jti="testdpop3"))

,jti


# Test Drive

In [16]:
Create(RefreshTokenModel(jti="test1", access_token="access1", refresh_token="refresh1", client_id="client1", exp=555, active=True, remark=""))

,Count
0,1


In [17]:
Read(RefreshTokenModel(jti="test1", access_token="access1", refresh_token="refresh1", client_id="client1", exp=555, active=True, remark=""))

,jti,access_token,client_id,exp,active,remark,refresh_token
0,test1,access1,client1,555,True,,refresh1


In [18]:
Read(RefreshTokenModel(client_id="client1"))

,jti,access_token,client_id,exp,active,remark,refresh_token
0,test1,access1,client1,555,True,,refresh1


In [19]:
Create(RefreshTokenModel(jti="test2", access_token="access2", refresh_token="refresh2", client_id="client1", exp=555, active=True, remark=""))

,Count
0,1


In [20]:
Create(RefreshTokenModel(jti="test3", access_token="access3", refresh_token="refresh3", client_id="client2", exp=555, active=True, remark=""))

,Count
0,1


In [21]:
Read(RefreshTokenModel(client_id="client1"))

,jti,access_token,client_id,exp,active,remark,refresh_token
0,test1,access1,client1,555,True,,refresh1
1,test2,access2,client1,555,True,,refresh2


In [22]:
existing = RefreshTokenModel(client_id="client1", access_token="access1")
new = RefreshTokenModel(active=False, remark="test update")
Update(existing, new)

,Count
0,1


In [23]:
Read(RefreshTokenModel(client_id="client1"))

,jti,access_token,client_id,exp,active,remark,refresh_token
0,test1,access1,client1,555,False,test update,refresh1
1,test2,access2,client1,555,True,,refresh2


In [24]:
existing = RefreshTokenModel(client_id="client1")
new = RefreshTokenModel(remark="test update more than one records")
Update(existing, new)

,Count
0,2


In [25]:
Read(RefreshTokenModel(client_id="client1"))

,jti,access_token,client_id,exp,active,remark,refresh_token
0,test1,access1,client1,555,False,test update more than one records,refresh1
1,test2,access2,client1,555,True,test update more than one records,refresh2


In [26]:
Read(RefreshTokenModel())

,jti,access_token,client_id,exp,active,remark,refresh_token
0,test1,access1,client1,555,False,test update more than one records,refresh1
1,test2,access2,client1,555,True,test update more than one records,refresh2
2,test3,access3,client2,555,True,,refresh3


In [27]:
ezo.db("SELECT * FROM dpopmodel;")

,jti
0,testdpop1


# Delete tables

In [28]:
delete_tables(tables)

Model: dpopmodel deleted successfully
Model: accesstokenmodel deleted successfully
Model: refreshtokenmodel deleted successfully
All tables deleted successfully


In [29]:
query = """\
SELECT * 
FROM information_schema.tables 
WHERE table_schema = 'main';
""".strip()
duck_engine(query, [])

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT


# Test Idea

In [30]:
schema_description = """\
Table: customer_tbl
Columns:
    - PassengerId (INTEGER): Passenger ID
    - Survived (INTEGER): Customer dead or alive status
    - Pclass (INTEGER): Ticket class that a customer bought
    - Name (STRING): Name of customer
    - Sex (STRING): Gender of customer includings male and female
    - Age (FLOAT): Age of customer
    - SibSp (INTEGER): Number of customer's siblings and spouses 
    - Parch (INTEGER): Number of customer's parents and children
    - Ticket (STRING): Ticket's code
    - Fare (FLOAT): Ticket's fee
    - Cabin (STRING): Place where a customer stayed
    - Embarked (STRING): Place where a customer got on board
"""

In [31]:
Field(description="Passenger unique identifier", default=None)

FieldInfo(annotation=NoneType, required=False, default=None, description='Passenger unique identifier')

In [ ]:
from ezorm.variables import EzORM, EzField

In [32]:
from ezorm.variables import EzORM, EzField
from typing import Optional, get_args

class customer_tbl(EzORM):
    """Information relevant to customer in the titanic transportation."""
    PassengerId:int = EzField(description="Passenger unique identifier", default=None)
    Survived:Optional[int] = EzField(description="Customer surival status: dead=0, alive=1", default=None)
    Pclass:int = EzField(description="Ticket class that a customer bought", default=None)
    Name:str = EzField(description="Name of customer", default=None)
    Sex:str = EzField(description="Biological sex contains only male and female", default=None)
    Age:float = EzField(description="Age of customer", default=None)
    SibSp:int = EzField(description="Number of customer's siblings and spouses", default=None)
    Parch:int = EzField(description="Number of customer's parents and children", default=None)
    Ticket:str = EzField(description="Ticket identifier", default=None)
    Fare:float = EzField(description="Fee of the ticket", default=None)
    Cabin:str = EzField(description="Place where a customer stayed", default=None)
    Embarked:str = EzField(description="Place where a customer got on board at the first place", default=None)

print(customer_tbl.to_prompt())
_ = customer_tbl.data_dict()

Table: customer_tbl
Description: Information relevant to customer in the titanic transportation.
Columns:
	- PassengerId (INTEGER): Passenger unique identifier
	- Survived (INTEGER): Customer surival status: dead=0, alive=1
	- Pclass (INTEGER): Ticket class that a customer bought
	- Name (STRING): Name of customer
	- Sex (STRING): Biological sex contains only male and female
	- Age (FLOAT): Age of customer
	- SibSp (INTEGER): Number of customer's siblings and spouses
	- Parch (INTEGER): Number of customer's parents and children
	- Ticket (STRING): Ticket identifier
	- Fare (FLOAT): Fee of the ticket
	- Cabin (STRING): Place where a customer stayed
	- Embarked (STRING): Place where a customer got on board at the first place
----------------------------------------------------------------------------------
Table: customer_tbl
Description: Information relevant to customer in the titanic transportation.
----------------------------------------------------------------------------------
| co

In [33]:
get_args(customer_tbl.model_fields['Survived'].annotation)

(int, NoneType)

In [34]:
from io import StringIO
import pandas as pd
import json

pd.read_json(StringIO(json.dumps(_['columns'])), orient="records")

,name,dtype,description
0,PassengerId,INTEGER,Passenger unique identifier
1,Survived,INTEGER,"Customer surival status: dead=0, alive=1"
2,Pclass,INTEGER,Ticket class that a customer bought
3,Name,TEXT,Name of customer
4,Sex,TEXT,Biological sex contains only male and female
5,Age,FLOAT,Age of customer
6,SibSp,INTEGER,Number of customer's siblings and spouses
7,Parch,INTEGER,Number of customer's parents and children
8,Ticket,TEXT,Ticket identifier
9,Fare,FLOAT,Fee of the ticket
